In [1]:
from langchain_ollama.llms import OllamaLLM
llm = OllamaLLM(model="llama3.2:1b")

In [2]:
from typing import Annotated
from typing_extensions import TypedDict

from langgraph.graph import StateGraph
from langgraph.graph.message import add_messages


class State(TypedDict):
    messages: Annotated[list, add_messages]


graph_builder = StateGraph(State)

def chatbot(state: State):
    return {"messages": [llm.invoke(state["messages"])]}


# The first argument is the unique node name
# The second argument is the function or object that will be called whenever
# the node is used.
graph_builder.add_node("chatbot", chatbot)
graph_builder.set_entry_point("chatbot")
graph_builder.set_finish_point("chatbot")
graph = graph_builder.compile()

def stream_graph_updates(user_input: str):
    for event in graph.stream({"messages": [("user", user_input)]}):
        for value in event.values():
            print("Assistant:", value["messages"][-1])

while True:
    try:
        user_input = input("User: ")
        print("User: " + user_input)
        if user_input.lower() in ["quit", "exit", "q"]:
            print("Goodbye!")
            break

        stream_graph_updates(user_input)
    except:
        # fallback if input() is not available
        user_input = input()
        print("User: " + user_input)
        stream_graph_updates(user_input)
        break

User: what is love?
Assistant: Love is a complex and multifaceted emotion that can be challenging to define, but I'll try to provide a general understanding of it.

Love encompasses a range of feelings and behaviors that are unique to each individual. It's a mix of emotions, such as affection, attachment, care, and passion, which can vary from person to person and even from one situation to another.

Some people believe that love is a strong feeling of attachment or connection with another person, often accompanied by a desire for mutual support, understanding, and intimacy. Others see it as a profound appreciation for the beauty and value of life itself, often leading to a deep sense of gratitude and devotion.

From a philosophical perspective, love has been described as a fundamental aspect of human nature, essential for personal growth, relationships, and community building. It's what drives us to care for others, to sacrifice our own needs for their benefit, and to strive for happi